# 7.2 Tool Use 实战 - OpenAI vs Anthropic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/07-function-calling/tool_use.ipynb)

**预计 API 费用: ~$0.02**

对比 OpenAI 和 Anthropic 的工具调用方式,实现统一接口。

In [ ]:
!pip install -q openai anthropic

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("请输入 OpenAI API Key: ")

if not os.environ.get("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass("请输入 Anthropic API Key (可选): ")

## 实验 1: OpenAI 并行工具调用

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

# 定义多个工具
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取城市天气",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "执行计算",
            "parameters": {
                "type": "object",
                "properties": {"expression": {"type": "string"}},
                "required": ["expression"]
            }
        }
    }
]

def get_weather(city: str):
    return {"city": city, "temperature": 15, "weather": "晴"}

def calculator(expression: str):
    return {"result": eval(expression)}

# 测试并行调用
messages = [{"role": "user", "content": "北京天气如何?顺便帮我算 15*7"}]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

message = response.choices[0].message
messages.append(message)

print(f"LLM 调用了 {len(message.tool_calls)} 个工具:\n")

for tool_call in message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    
    print(f"🔧 {name}({args})")
    
    if name == "get_weather":
        result = get_weather(**args)
    else:
        result = calculator(**args)
    
    print(f"   结果: {result}\n")
    
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result, ensure_ascii=False)
    })

final = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(f"✅ 最终回答:\n{final.choices[0].message.content}")

## 实验 2: Anthropic Tool Use

In [ ]:
from anthropic import Anthropic
import json
import os

if not os.environ.get("ANTHROPIC_API_KEY"):
    print("⚠️  未配置 ANTHROPIC_API_KEY,跳过此实验")
else:
    client = Anthropic()
    
    # Anthropic 的工具定义格式
    tools = [
        {
            "name": "get_weather",
            "description": "获取城市天气",
            "input_schema": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            }
        },
        {
            "name": "calculator",
            "description": "执行计算",
            "input_schema": {
                "type": "object",
                "properties": {"expression": {"type": "string"}},
                "required": ["expression"]
            }
        }
    ]
    
    messages = [{"role": "user", "content": "北京天气如何?顺便帮我算 15*7"}]
    
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        tools=tools,
        messages=messages
    )
    
    print(f"Stop reason: {response.stop_reason}\n")
    
    tool_results = []
    for block in response.content:
        if block.type == "tool_use":
            print(f"🔧 {block.name}({block.input})")
            
            if block.name == "get_weather":
                result = get_weather(**block.input)
            else:
                result = calculator(**block.input)
            
            print(f"   结果: {result}\n")
            
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": block.id,
                "content": json.dumps(result, ensure_ascii=False)
            })
    
    # 返回结果给 Claude
    messages.append({"role": "assistant", "content": response.content})
    messages.append({"role": "user", "content": tool_results})
    
    final = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        tools=tools,
        messages=messages
    )
    
    print(f"✅ 最终回答:\n{final.content[0].text}")

## 实验 3: 统一适配器接口

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Callable
import json

class ToolUseAdapter(ABC):
    """工具调用适配器基类"""
    
    @abstractmethod
    def call_with_tools(self, messages: List[Dict], tools: List[Dict]):
        pass
    
    @abstractmethod
    def execute_tools(self, response, functions: Dict[str, Callable]) -> List[Dict]:
        pass

class OpenAIAdapter(ToolUseAdapter):
    def __init__(self, client):
        self.client = client
    
    def call_with_tools(self, messages, tools):
        return self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )
    
    def execute_tools(self, response, functions):
        results = []
        message = response.choices[0].message
        
        if message.tool_calls:
            for tc in message.tool_calls:
                func = functions[tc.function.name]
                args = json.loads(tc.function.arguments)
                result = func(**args)
                
                results.append({
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": json.dumps(result, ensure_ascii=False)
                })
        
        return results

# 测试
from openai import OpenAI

adapter = OpenAIAdapter(OpenAI())

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取天气",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            }
        }
    }
]

messages = [{"role": "user", "content": "北京天气?"}]

response = adapter.call_with_tools(messages, tools)
tool_results = adapter.execute_tools(response, {"get_weather": get_weather})

print(f"执行了 {len(tool_results)} 个工具")
print(json.dumps(tool_results, indent=2, ensure_ascii=False))

## 关键要点总结

1. **OpenAI**: `tool_calls` 响应,`role: tool` 返回
2. **Anthropic**: `content` 数组,`tool_result` 类型
3. **并行调用**: 两者都支持一次返回多个工具调用
4. **统一接口**: 用适配器模式屏蔽差异

---

**下一步**: 学习 [7.3 工具编排与回退](./orchestration.ipynb)